<a href="https://colab.research.google.com/github/lblum95/AML/blob/master/task3/voting_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import  HistGradientBoostingClassifier, RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [26]:
x_train = pd.read_csv("data/X_train_features.csv", index_col=0, header=0, low_memory=False)
y_train = pd.read_csv("data/y_train.csv", index_col=0, header=0)
x_test = pd.read_csv("data/X_test_features.csv", index_col=0, header=0, low_memory=False)

normal_features=x_train.iloc[:,0:x_train.columns.get_loc("J_L/H")]
tescht=x_test.iloc[:,0:x_test.columns.get_loc("J_L/H")]
imputer=SimpleImputer(strategy='median')

scaler = StandardScaler()
norm=normal_features.drop(['HRV_ULF','HRV_VLF','HRV_LF','HRV_LFHF','HRV_LFn'], axis=1)
tescht=tescht.drop(['HRV_ULF','HRV_VLF','HRV_LF','HRV_LFHF','HRV_LFn'], axis=1)
norm=norm.replace(np.inf, 10)
tescht=tescht.replace(np.inf, 10)
norm=norm.replace(-np.inf, -10)
tescht=tescht.replace(-np.inf, -10)
norm['y']=y_train
y_tr=norm['y']
selected=norm.drop(['y'], axis=1)
#selected=imputer.fit_transform(norm)

In [49]:
from sklearn.pipeline import Pipeline
def get_model():
  estimators = [('l2', HistGradientBoostingClassifier(l2_regularization=3, max_leaf_nodes=None,max_depth=None)),
                ('full_steam', HistGradientBoostingClassifier(max_leaf_nodes=None,max_depth=None, min_samples_leaf=10)),
                ('20', HistGradientBoostingClassifier(max_depth=None, min_samples_leaf=20)),
                ('rfc',Pipeline([('scaler', SimpleImputer(strategy ='median')),('svc', RandomForestClassifier(n_estimators=500,min_samples_leaf=2))])),
                ('rf_feat',Pipeline([('scaler', SimpleImputer(strategy='median')), ('svc', RandomForestClassifier(n_estimators=100,min_samples_split=2))]))]
  clf = VotingClassifier(estimators=estimators,voting='soft',flatten_transform=True,n_jobs=-1)
  return clf
skf1=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index,test_index in skf1.split(selected,y_tr):
  X_train = selected.iloc[train_index]
  X_test = selected.iloc[test_index]
  Y_train =y_tr[train_index]
  Y_test = y_tr[test_index]
  model=get_model()
  model.fit(X_train,Y_train)
  Y_pred=model.predict(X_test)
  print(confusion_matrix(Y_test, Y_pred))
  print('Score')
  print(f1_score(Y_test, Y_pred, average='micro'))
  print()
  print()

[[568   2  34   2]
 [  8  61  19   1]
 [ 89   8 197   1]
 [ 11   4   4  15]]
Score
0.8212890625


[[565   1  36   4]
 [  8  65  16   0]
 [ 85   9 197   4]
 [  7   0   6  21]]
Score
0.828125


[[572   3  30   1]
 [  5  59  19   5]
 [ 83  10 199   3]
 [  7   0   4  23]]
Score
0.833822091886608


[[575   1  29   1]
 [  1  67  20   0]
 [ 95  16 184   0]
 [ 17   1   1  15]]
Score
0.8220918866080157


[[558   2  40   6]
 [  5  67  16   1]
 [ 82  11 196   5]
 [ 13   0   4  17]]
Score
0.8191593352883676




In [43]:
def get_model():
  estimators = [('l2', HistGradientBoostingClassifier(l2_regularization=3, max_leaf_nodes=None,max_depth=None,min_samples_leaf=10)),
                ('full_steam', HistGradientBoostingClassifier(max_leaf_nodes=None,max_depth=None, min_samples_leaf=10)),
                ('20', HistGradientBoostingClassifier(max_depth=None, min_samples_leaf=20)),
                ('rfc',Pipeline([('scaler', SimpleImputer(strategy ='median')),('svc', RandomForestClassifier(n_estimators=500))])),
                ('rf_feat',Pipeline([('scaler', SimpleImputer(strategy='median')), ('svc', RandomForestClassifier(n_estimators=100,min_samples_split=2))]))]
  clf = VotingClassifier(estimators=estimators,voting='soft',flatten_transform=True,n_jobs=-1)
  return clf
skf1=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index,test_index in skf1.split(selected,y_tr):
  X_train = selected.iloc[train_index]
  X_test = selected.iloc[test_index]
  Y_train =y_tr[train_index]
  Y_test = y_tr[test_index]
  model=get_model()
  model.fit(X_train,Y_train)
  Y_pred=model.predict(X_test)
  print(confusion_matrix(Y_test, Y_pred))
  print('Score')
  print(f1_score(Y_test, Y_pred, average='micro'))
  print()
  print()

[[567   3  34   2]
 [  7  61  20   1]
 [ 88   6 200   1]
 [ 10   4   4  16]]
Score
0.82421875


[[566   1  34   5]
 [  8  63  18   0]
 [ 84   9 196   6]
 [  6   0   5  23]]
Score
0.828125


[[573   3  29   1]
 [  5  59  19   5]
 [ 84  10 199   2]
 [  7   0   4  23]]
Score
0.8347996089931574


[[573   0  32   1]
 [  2  66  20   0]
 [ 92  14 188   1]
 [ 17   1   1  15]]
Score
0.823069403714565


[[556   2  42   6]
 [  5  67  16   1]
 [ 78  12 199   5]
 [ 13   0   3  18]]
Score
0.8211143695014664




In [45]:
def get_model():
  estimators = [('l2', HistGradientBoostingClassifier(l2_regularization=3, max_leaf_nodes=None,max_depth=None,min_samples_leaf=10)),
                ('full_steam', HistGradientBoostingClassifier(max_leaf_nodes=None,max_depth=None, min_samples_leaf=10)),
                ('20', HistGradientBoostingClassifier(max_depth=None, min_samples_leaf=20)),
                ('rfc',Pipeline([('scaler', SimpleImputer(strategy ='median')),('svc', RandomForestClassifier(n_estimators=500))])),
                ('rf_feat',Pipeline([('scaler', SimpleImputer(strategy='median')), ('svc', RandomForestClassifier(n_estimators=100,min_samples_split=2))]))]
  clf = VotingClassifier(estimators=estimators,voting='soft',flatten_transform=True,n_jobs=-1)
  return clf
model = get_model()
model.fit(selected,y_tr)
y_tescht=model.predict(tescht)
df = pd.DataFrame(y_tescht)
df.to_csv('data/vote.csv', header=['y'], index_label='id')